In [1]:
function rotation_matrix(axis, rotation_angle)

  #=
   =======================================================================
  Creates a rotation matrix for a rotation around an arbitrary axis as
  described by the Euler-Rodrigues formula

  NOTE: Use this matrix to explicitly rotate about an arbitrary axis. This
  matrix is more easily used to transform  between Aki-Richards convention
  and standard RH coordinate systems than the other rotationmat3D function
   =======================================================================

  =#

    axis_normalized = axis/sqrt(vecdot(axis, axis))
    a = cos(rotation_angle/2)
    b = -axis_normalized[1]*sin(rotation_angle/2)
    c = -axis_normalized[2]*sin(rotation_angle/2)
    d = -axis_normalized[3]*sin(rotation_angle/2)
    rot_mat = [a^2+b^2-c^2-d^2 2*(b*c-a*d) 2*(b*d+a*c);
               2*(b*c+a*d) a^2+c^2-b^2-d^2 2*(c*d-a*b);
               2*(b*d-a*c) 2*(c*d+a*b) (a^2+d^2-b^2-c^2)]

    return rot_mat
end

rotation_matrix (generic function with 1 method)

In [2]:
function stress_rotation(phi, theta, rho)
    
    Rstrike = rotation_matrix([0, 0, 1], phi)
    yp = Rstrike*[0, -1, 0]
    Rdip = rotation_matrix(yp, theta)
    Ri = Rdip*Rstrike
    xpp = Ri*[1, 0, 0]
    Rrho = rotation_matrix(xpp, rho)
    Rstress = Rrho*Ri
    
    return Rstress
    
end

stress_rotation (generic function with 1 method)

In [3]:
function fault_rotation(strike, dip)
    
    Rstrike = rotation_matrix([0, 0, 1], strike)
    yp = Rstrike*[0, -1, 0]
    Rdip = rotation_matrix(yp, dip)
    Rfault = Rdip*Rstrike
    
    return Rfault
    
end
    
    

fault_rotation (generic function with 1 method)

In [4]:
function ordered_eigensystem(M)
    
    vals, vecs = eig(M)
    idx_sorted = sortperm(vals)
    vals_sorted = vals[idx_sorted]
    vecs_sorted = vecs[:, idx_sorted]
    
    return vals_sorted, vecs_sorted
    
end

ordered_eigensystem (generic function with 1 method)

In [28]:
pstress = [-1, -1/2, -1/4];
sangle = [0, 0, 0];
fangle = [0, pi/4, 0];

phiF   = fangle[1]
thetaF = fangle[2]
rakeF  = fangle[3]

0.0

In [29]:
if size(pstress) == (3,3)
    
        stressDirection = pstress;

    else
        #sangle = [Stress_strike_MCS, Stress_dip_MCS, Stress_dip_ICS]
    
    
        S1 = pstress[1];
        S2 = pstress[2];
        S3 = pstress[3];
        phi_MCS   = sangle[1];
        theta_MCS = sangle[2];
        rho = sangle[3];

        #define the principal stress tensor
        Sprincipal = [S1 0.0 0.0;
                      0.0 S2 0.0;
                      0.0 0.0 S3];

end

3×3 Array{Float64,2}:
 -1.0   0.0   0.0 
  0.0  -0.5   0.0 
  0.0   0.0  -0.25

In [31]:
Rstress = stress_rotation(phi_MCS, theta_MCS, rho)
Rfault = fault_rotation(phiF, thetaF)

S_geographic = Rstress'*Sprincipal*Rstress
S_fault = Rfault*S_geographic*Rfault'

SigmaN = S_fault[3,3]
Tau_rake_parallel = S_fault[2,3]
Tau_rake_perpendicular = S_fault[1,3]
TauDip = Tau_rake_parallel 
TauStrike = Tau_rake_perpendicular 
Tau_rake_parallel 
RakeCalc = atan2(TauStrike, TauDip);

println(RakeCalc*180/pi)





90.0
